In [1]:
from experiment import ResultsViewer
from scipy.io import savemat, loadmat
import numpy as np
from torch import load
from data import Data
from model import Model
import json
import yaml
import os

paths = [
    "experiments/FIXED/w_v_low_rank_regularized_10_datasets",
    "experiments/FIXED/w_v_low_rank_regularized_10_datasets_123",
    "experiments/FIXED/w_v_low_rank_regularized_10_datasets_45",
    "experiments/FIXED/w_v_low_rank_regularized_10_datasets_67",
    "experiments/FIXED/w_v_low_rank_regularized_10_datasets_89"
]

newdir = "experiments/Jun4/w_v_low_rank_fixed"
os.makedirs(newdir, exist_ok=True)

def get_data_model(foo):
    data = Data(foo['data']['D'])
    data.load(foo['data']['data_id'])
    model = Model(foo['data']['D'], **foo['model'])
    model.load_state_dict(load(foo['state_dict_path']))
    return data, model

def rewrite_model(foo, model):
    path = newdir + "/" + foo['matlab_path'].split("/")[-1] + '-fixed'
    mat = model.state_dict_matlab()
    savemat(path, mat)
    return path, mat

def mse_regterm(data, model):
    x, y = data.training_data()
    pred = model.predict(x)
    mse = (np.sum(np.power(pred-y,2)) / pred.size).item()
    rterm = model.regularization_value_metric().detach().numpy().item()
    return mse, rterm

def rewrite_all_in_dir(paths):
    fixed_res = []
    for path in paths:
        results = ResultsViewer(path)
        for i in range(len(results)):
            data, model = get_data_model(results[i])
            path, mat = rewrite_model(results[i], model)
            by_epoch = results[i]['report']['Sparsity by Epoch']
            mse, term = mse_regterm(data, model)
            new_res = {
                "matlab_path": results[i]['matlab_path'].split("/")[-1] + '-fixed',
                "regularization_value": term, 
                "MSE": mse,
                "final_sparsity": results[i]['report']['Sparsity'],
                "sparsity_by_epoch": results[i]['report']['Sparsity by Epoch'],
            }
            new_res.update(results[i]['model'].copy())
            fixed_res.append(new_res)
    fixed_path = newdir + "/results-rahul.json"
    with open(fixed_path, "w") as fp:
        fp.write(json.dumps(fixed_res))
    return fixed_path, fixed_res

In [2]:
path, res = rewrite_all_in_dir(paths)